<a href="https://colab.research.google.com/github/andervies/divic-corp-machine-learning-course/blob/main/assignment27/Tensorflow_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem One: Looking back on scratch

1. **Initialize the Weights:**
    - I used different initializers like Xavier, He, and SimpleInitializer to initialize the weights and biases for each layer.

2. **Epoch Loop:**
    - I implemented a loop to run the training process for a specified number of epochs.

3. **Batch Processing:**
    - I created mini-batches from the training data to process the data in smaller chunks.

4. **Forward Propagation:**
    - I implemented the forward pass for each layer type, including Convolutional, MaxPooling, Flatten, and Fully Connected layers.

5. **Activation Functions:**
    - I used activation functions like ReLU, Sigmoid, and Tanh in the forward and backward passes.

6. **Loss Calculation:**
    - I calculated the loss using functions like Softmax with Cross-Entropy Loss.

7. **Backward Propagation:**
    - I implemented the backward pass to compute gradients for each layer.

8. **Weight Update:**
    - I updated the weights and biases using optimization algorithms like SGD and AdaGrad.

9. **Data Preprocessing:**
    - I loaded, normalized, and reshaped the dataset. I also performed one-hot encoding of the labels.

10. **Model Evaluation:**
    - I implemented functions to predict and evaluate the model's performance using metrics like accuracy.

## Problem Two: Consider the correspondence between scratch and TensorFlow

1. **Load and Prepare Data:**
    - **Scratch:** Load dataset, preprocess, and split into training, validation, and test sets. Implement mini-batch processing.
    - **TensorFlow:** Load dataset, preprocess, and split into training, validation, and test sets. Implement mini-batch processing using an iterator class.

2. **Define Hyperparameters:**
    - **Scratch:** Set hyperparameters such as learning rate, batch size, number of epochs, and number of nodes in each layer.
    - **TensorFlow:** Define hyperparameters like learning rate, batch size, number of epochs, and number of nodes in each layer using variables.

3. **Initialize Weights and Biases:**
    - **Scratch:** Manually initialize weights and biases for each layer using custom functions or classes.
    - **TensorFlow:** Use TensorFlow’s built-in methods to initialize weights and biases, such as `tf.Variable` and `tf.random_normal`.

4. **Build the Model:**
    - **Scratch:** Manually implement forward propagation through each layer, applying activation functions.
    - **TensorFlow:** Define the model architecture using TensorFlow operations like `tf.add`, `tf.matmul`, and activation functions such as `tf.nn.relu`.

5. **Define Loss and Optimizer:**
    - **Scratch:** Implement loss function and manually compute gradients for backpropagation.
    - **TensorFlow:** Use TensorFlow’s built-in loss functions (e.g., `tf.nn.sigmoid_cross_entropy_with_logits`) and optimizers (e.g., `tf.train.AdamOptimizer`).

6. **Training Loop:**
    - **Scratch:** Write a loop to iterate over epochs and mini-batches, updating weights using computed gradients.
    - **TensorFlow:** Use TensorFlow’s session to run the computation graph, executing the training operation for each mini-batch and epoch.

7. **Evaluate Model:**
    - **Scratch:** Manually compute accuracy and other metrics by comparing predictions with actual labels.
    - **TensorFlow:** Use TensorFlow operations to compute accuracy and other metrics, such as `tf.reduce_mean` and `tf.cast`.

**Summary:**
- TensorFlow automates many steps involved in implementing deep learning models, such as weight initialization, forward propagation, loss calculation, and backpropagation.
- It provides a structured and efficient way to build, train, and evaluate models using its computation graph and built-in functions.
- This abstraction allows for easier experimentation and scaling of models compared to manual implementation from scratch.

In [ ]:
tf = tf.compat.v1
tf.disable_eager_execution()

In [ ]:
"""
Binary classification of Iris dataset using neural network implemented in TensorFlow
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

tf = tf.compat.v1
tf.disable_eager_execution()

tf.test.gpu_device_name()
"""
Don't forget when you change the version of tensorflow to the 1.x series.
Install the GPU with "!pip install tensorflow-gpu==1.14.0".
tf.test.gpu_device_name()でGPUの設定状態を確認し、認識されるかを確認します。
If successful, a log is output; if not recognized, nothing is output。
"""

#Load dataset
df = pd.read_csv("drive/MyDrive/assignment27/Iris.csv")

#Condition extraction from data frame
df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

# Convert to NumPy array
X = np.array(X)
y = np.array(y)
#Convert labels to numbers
y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)[:, np.newaxis]

#Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Further split into train and val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    Iterator to get a mini-batch

    Parameters
    ----------
    X : The following forms of ndarray, shape (n_samples, n_features)
      Training data
    y : The following form of ndarray, shape (n_samples, 1)
      Correct answer value
    batch_size : int
      Batch size
    seed : int
      NumPy random seed
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int64)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

#Determine the shape of the argument to be passed to the calculation graph
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    #Declaration of weights and biases
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.add and + are equivalent
    return layer_output

#Read network structure
logits = example_net(X)

# Objective function
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
#Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Estimated result
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
#Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialization of variable
init = tf.global_variables_initializer()


#Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        #Loop for each epoch
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int64)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}".format(epoch, total_loss, val_loss, acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 7.0241, val_loss : 67.6860, acc : 0.375
Epoch 1, loss : 3.4241, val_loss : 23.4026, acc : 0.312
Epoch 2, loss : 1.9387, val_loss : 11.6681, acc : 0.375
Epoch 3, loss : 2.0917, val_loss : 13.1400, acc : 0.312
Epoch 4, loss : 1.7685, val_loss : 17.7284, acc : 0.312
Epoch 5, loss : 1.6097, val_loss : 12.9607, acc : 0.312
Epoch 6, loss : 1.4402, val_loss : 10.0593, acc : 0.312
Epoch 7, loss : 1.3704, val_loss : 9.4797, acc : 0.312
Epoch 8, loss : 1.2536, val_loss : 9.8518, acc : 0.312
Epoch 9, loss : 1.1476, val_loss : 8.5670, acc : 0.375
Epoch 10, loss : 1.0930, val_loss : 8.0430, acc : 0.375
Epoch 11, loss : 1.0412, val_loss : 7.8791, acc : 0.375
Epoch 12, loss : 0.9804, val_loss : 7.1233, acc : 0.375
Epoch 13, loss : 0.9326, val_loss : 6.7908, acc : 0.375
Epoch 14, loss : 0.8792, val_loss : 6.2492, acc : 0.375
Epoch 15, loss : 0.8304, val_loss : 5.7680, acc : 0.375
Epoch 16, loss : 0.7835, val_loss : 5.2886, acc : 0.438
Epoch 17, loss : 0.7384, val_loss : 4.8037, acc : 0

## Problem Three: Create a model of Iris using all three types of objective variables

In [ ]:
# Load dataset
df = pd.read_csv("Iris.csv")

# Filter data for three-class classification
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

# Convert to NumPy array
X = np.array(X)
y = np.array(pd.get_dummies(y))

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

class GetMiniBatch:
    """
    Iterator to get a mini-batch

    Parameters
    ----------
    X : ndarray, shape (n_samples, n_features)
      Training data
    y : ndarray, shape (n_samples, n_classes)
      Correct answer values
    batch_size : int
      Batch size
    seed : int
      NumPy random seed
    """
    def __init__(self, X, y, batch_size=10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0] / self.batch_size).astype(np.int64)

    def __len__(self):
        return self._stop

    def __getitem__(self, item):
        p0 = item * self.batch_size
        p1 = item * self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]

    def __iter__(self):
        self._counter = 0
        return self

    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter * self.batch_size
        p1 = self._counter * self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_classes = y_train.shape[1]

# Define placeholders
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def neural_net(x):
    """
    Simple 3-layer neural network
    """
    tf.random.set_random_seed(0)
    # Declaration of weights and biases
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    output_layer = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])  # tf.add and + are equivalent
    return output_layer

# Read network structure
logits = neural_net(X)

# Objective function
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Estimated result
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))

# Indicator value calculation
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialization of variables
init = tf.global_variables_initializer()

# Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # Loop for each epoch
        total_batch = np.ceil(X_train.shape[0] / batch_size).astype(np.int64)
        total_loss = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print(f"Epoch {epoch}, loss: {total_loss:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.3f}")

    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print(f"Test accuracy: {test_acc:.3f}")


Epoch 0, loss: 20.5204, val_loss: 135.0502, val_acc: 0.333
Epoch 1, loss: 15.0845, val_loss: 98.2559, val_acc: 0.333
Epoch 2, loss: 10.3703, val_loss: 61.5122, val_acc: 0.533
Epoch 3, loss: 6.0867, val_loss: 34.2166, val_acc: 0.267
Epoch 4, loss: 2.8178, val_loss: 10.5846, val_acc: 0.533
Epoch 5, loss: 0.5077, val_loss: 1.2543, val_acc: 0.867
Epoch 6, loss: 0.0851, val_loss: 0.1723, val_acc: 0.933
Epoch 7, loss: 0.0546, val_loss: 0.1215, val_acc: 0.933
Epoch 8, loss: 0.0524, val_loss: 0.0879, val_acc: 0.933
Epoch 9, loss: 0.0507, val_loss: 0.0755, val_acc: 0.933
Epoch 10, loss: 0.0496, val_loss: 0.0678, val_acc: 0.933
Epoch 11, loss: 0.0487, val_loss: 0.0610, val_acc: 0.933
Epoch 12, loss: 0.0478, val_loss: 0.0545, val_acc: 0.933
Epoch 13, loss: 0.0469, val_loss: 0.0487, val_acc: 0.933
Epoch 14, loss: 0.0463, val_loss: 0.0435, val_acc: 1.000
Epoch 15, loss: 0.0455, val_loss: 0.0375, val_acc: 1.000
Epoch 16, loss: 0.0446, val_loss: 0.0337, val_acc: 1.000
Epoch 17, loss: 0.0436, val_loss

## Problem Four: Create a model of House Prices

In [ ]:
# Load dataset
house_data = pd.read_csv("train.csv")

# Select relevant columns
X = house_data[["GrLivArea", "YearBuilt"]]
y = house_data["SalePrice"]

# Convert to NumPy arrays
X = np.array(X)
y = np.array(y).reshape(-1, 1)

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)


# Hyperparameter settings
learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_output = 1

# Define placeholders
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])

# Train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def regression_net(x):
    """
    Simple 3-layer neural network for regression
    """
    tf.random.set_random_seed(0)
    # Declaration of weights and biases
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_output]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_output]))
    }

    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    output_layer = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    return output_layer

# Read network structure
logits = regression_net(X)

# Objective function (Mean Squared Error)
loss_op = tf.reduce_mean(tf.square(logits - Y))

# Optimization method
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Initialization of variables
init = tf.global_variables_initializer()

# Run calculation graph
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # Loop for each epoch
        total_batch = np.ceil(X_train.shape[0] / batch_size).astype(np.int64)
        total_loss = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Loop for each mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= n_samples
        val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        print(f"Epoch {epoch}, loss: {total_loss:.4f}, val_loss: {val_loss:.4f}")

    test_loss = sess.run(loss_op, feed_dict={X: X_test, Y: y_test})
    print(f"Test loss: {test_loss:.4f}")


Epoch 0, loss: 23732756448.0000, val_loss: 3691986944.0000
Epoch 1, loss: 5639454295.5000, val_loss: 3748113920.0000
Epoch 2, loss: 5462861265.5000, val_loss: 3841552384.0000
Epoch 3, loss: 5396506829.5000, val_loss: 3910583296.0000
Epoch 4, loss: 5381580262.5000, val_loss: 3942595840.0000
Epoch 5, loss: 5376558308.5000, val_loss: 3966002944.0000
Epoch 6, loss: 5373939226.0000, val_loss: 3979547136.0000
Epoch 7, loss: 5372898390.0000, val_loss: 3984245504.0000
Epoch 8, loss: 5372443442.0000, val_loss: 3982286848.0000
Epoch 9, loss: 5372406419.0000, val_loss: 3985517056.0000
Epoch 10, loss: 5372446631.0000, val_loss: 3982362368.0000
Epoch 11, loss: 5372699777.5000, val_loss: 3982856960.0000
Epoch 12, loss: 5373366258.5000, val_loss: 3980841984.0000
Epoch 13, loss: 5373516686.0000, val_loss: 3978752512.0000
Epoch 14, loss: 5374128545.5000, val_loss: 3976596224.0000
Epoch 15, loss: 5374475719.5000, val_loss: 3973945600.0000
Epoch 16, loss: 5374418526.5000, val_loss: 3971655424.0000
Epoch 

## Problem Five: Create a model of MNIST

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()



# Reshape and normalize pixel values
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32') / 255.0

# Split into train, validation, and test sets (after reshaping)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


# One-hot encode labels (if needed for your model)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=10) # One-hot encode validation labels

# Parameters
learning_rate = 0.001
batch_size = 100
num_epochs = 10

n_input = 28 * 28  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Define placeholders
X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, n_classes])


# Train mini batch iterator
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)




def conv_net(x):
    # Conv Layer 1
    conv1 = tf.layers.conv2d(inputs=x, filters=32, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
    # Pooling Layer 1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Conv Layer 2
    conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
    # Pooling Layer 2
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Flatten tensor
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

    # Dense Layer
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    # Dropout
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=True)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=n_classes)
    return logits

# Build the model
logits = conv_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables
init = tf.global_variables_initializer()

# Train the model
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        total_loss = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # Reshape mini-batch to match placeholder shape
            mini_batch_x = mini_batch_x.reshape(-1, 28, 28, 1)  # Reshape mini-batch
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
        total_loss /= len(get_mini_batch_train)
        # Reshape validation data to match placeholder shape
        X_val_reshaped = X_val.reshape(-1, 28, 28, 1)  # Reshape validation data
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val_reshaped, Y: y_val}) # Use reshaped validation data
        print(f"Epoch {epoch + 1}, loss: {total_loss:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.3f}")

    # Evaluate on test data
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print(f"Test Accuracy: {test_acc:.3f}")

<ipython-input-19-a8b428b62df0>:40: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  conv1 = tf.layers.conv2d(inputs=x, filters=32, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
<ipython-input-19-a8b428b62df0>:42: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
<ipython-input-19-a8b428b62df0>:45: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
<ipython-input-19-a8b428b62df0>:47: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  poo

Epoch 1, loss: 0.1410, val_loss: 0.0577, val_acc: 0.982
Epoch 2, loss: 0.0381, val_loss: 0.0492, val_acc: 0.986
Epoch 3, loss: 0.0241, val_loss: 0.0460, val_acc: 0.986
Epoch 4, loss: 0.0165, val_loss: 0.0368, val_acc: 0.988
Epoch 5, loss: 0.0118, val_loss: 0.0380, val_acc: 0.990
Epoch 6, loss: 0.0083, val_loss: 0.0415, val_acc: 0.990
Epoch 7, loss: 0.0077, val_loss: 0.0359, val_acc: 0.991
Epoch 8, loss: 0.0064, val_loss: 0.0573, val_acc: 0.987
Epoch 9, loss: 0.0058, val_loss: 0.0428, val_acc: 0.989
Epoch 10, loss: 0.0044, val_loss: 0.0421, val_acc: 0.990
Test Accuracy: 0.992
